In [4]:
#최종
import pandas as pd
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.units import inch
from reportlab.lib.enums import TA_JUSTIFY
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import os
import re
import platform
from datetime import datetime
# 파일 경로 설정
EXCEL_PATH = "/Users/a/Desktop/Work/Blog/2024 Adsense/wp/post/post_list.xlsx"
OUTPUT_DIR = "/Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/"
# 오늘 날짜 문자열 생성 (YYYY-MM-DD 형식)
today_str = datetime.now().strftime('%Y%m%d')
today_dir = os.path.join(OUTPUT_DIR, today_str)

# 오늘 날짜 폴더가 없으면 생성
if not os.path.exists(today_dir):
    os.makedirs(today_dir)
    print(f"오늘 날짜({today_str}) 폴더 생성 완료")




# 출력 디렉토리가 없으면 생성
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# 시스템에 따른 폰트 경로 설정
print("시스템 폰트 검색 중...")
font_registered = False

# macOS의 경우
if platform.system() == 'Darwin':
    possible_fonts = [
        '/System/Library/Fonts/AppleSDGothicNeo.ttc',
        '/System/Library/Fonts/AppleGothic.ttf',
        '/System/Library/Fonts/Supplemental/AppleGothic.ttf',
        '/Library/Fonts/AppleGothic.ttf',
        '/Library/Fonts/NanumGothic.ttf',
        '/System/Library/Fonts/STHeiti Light.ttc',
        '/System/Library/Fonts/STHeiti Medium.ttc'
    ]
    
    for font_path in possible_fonts:
        if os.path.exists(font_path):
            try:
                # TTF 폰트 등록
                print(f"폰트 발견: {font_path}")
                if font_path.endswith('.ttc'):
                    # TTC(TrueType Collection) 파일 처리
                    pdfmetrics.registerFont(TTFont('KoreanFont', font_path, subfontIndex=0))
                else:
                    pdfmetrics.registerFont(TTFont('KoreanFont', font_path))
                font_registered = True
                print(f"폰트 등록 성공: {font_path}")
                break
            except Exception as e:
                print(f"폰트 등록 실패({font_path}): {e}")
                continue
# Windows의 경우
elif platform.system() == 'Windows':
    possible_fonts = [
        'C:\\Windows\\Fonts\\malgun.ttf',
        'C:\\Windows\\Fonts\\gulim.ttc',
        'C:\\Windows\\Fonts\\batang.ttc',
        'C:\\Windows\\Fonts\\NanumGothic.ttf'
    ]
    
    for font_path in possible_fonts:
        if os.path.exists(font_path):
            try:
                pdfmetrics.registerFont(TTFont('KoreanFont', font_path))
                font_registered = True
                print(f"폰트 등록 성공: {font_path}")
                break
            except Exception as e:
                print(f"폰트 등록 실패({font_path}): {e}")
                continue

# 폰트 등록에 실패한 경우 경고
if not font_registered:
    print("경고: 한글 폰트를 등록할 수 없었습니다. 한글이 제대로 표시되지 않을 수 있습니다.")
    # 기본 폰트 설정
    font_name = 'Helvetica'
else:
    font_name = 'KoreanFont'

# 스타일 설정
styles = getSampleStyleSheet()
styles.add(ParagraphStyle(
    name='Normal_Justified',
    parent=styles['Normal'],
    fontName=font_name,
    alignment=TA_JUSTIFY,
    firstLineIndent=20,
    spaceBefore=12,
    spaceAfter=12,
    leading=14  # 줄 간격
))

# 엑셀 파일 로드
try:
    print(f"엑셀 파일 로드 중: {EXCEL_PATH}")
    df = pd.read_excel(EXCEL_PATH)
    print(f"총 {len(df)} 개의 행이 로드되었습니다.")
except Exception as e:
    print(f"엑셀 파일 로드 실패: {e}")
    import sys
    sys.exit(1)

# 각 행에 대해 PDF 생성
for index, row in df.iterrows():
    try:
        title = str(row['title']).strip()
        body = str(row['body'])
        # HTML 태그 제거 (<h2> 및 </h2>)
        body = body.replace('<h3>', '').replace('</h3>', '').replace('#', '').replace('**', '')    
        # 디버깅 정보 출력
        print(f"\n처리 중인 행 {index+1}/{len(df)} - 제목: {title[:30]}...")
        
        # 파일명에 사용할 수 없는 문자 제거
        safe_title = re.sub(r'[\\/*?:"<>|]', '', title)
        
        # 출력 파일 경로 (날짜 폴더 안에 저장)
        output_path = os.path.join(today_dir, f"{safe_title}_대표 기도문 모음 나눔터.pdf")

        # PDF 문서 생성
        doc = SimpleDocTemplate(
            output_path,
            pagesize=A4,
            rightMargin=72,  # 1 inch
            leftMargin=72,
            topMargin=72,
            bottomMargin=72
        )
        
        # 내용을 담을 리스트
        story = []
        
        # 제목 추가 (더 크고 굵게)
        title_style = ParagraphStyle(
            name='Title',
            parent=styles['Title'],
            fontName=font_name,
            fontSize=16,
            spaceAfter=24
        )
        
        # HTML 예약 문자 처리
        title_safe = title.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
        story.append(Paragraph(title_safe, title_style))
        story.append(Spacer(1, 0.2 * inch))
        
        # <h2> 태그를 찾는 정규 표현식
        h2_pattern = re.compile(r'<h2>(.*?)</h2>')
        
        # 구분선 스타일 정의
        separator_style = ParagraphStyle(
            name='Separator',
            parent=styles['Normal'],
            fontName=font_name,
            alignment=TA_JUSTIFY
        )

        # 본문 라인을 배열로 변환
        body_lines = body.split('\n')
        processed_lines = []
        
        # 각 라인을 검사하고 <h2> 태그 앞에 구분선 추가
        for i, line in enumerate(body_lines):
            if '<h2>' in line:
                # <h2> 태그 앞에 구분선 추가
                processed_lines.append("---------------------------------------------------------------------")
            processed_lines.append(line)
        
        # 처리된 라인을 다시 문자열로 합치기
        body = '\n'.join(processed_lines)
        
        # <h2> 태그를 찾아 스타일 적용
        styled_body = ""
        for para in body.split('\n'):
            # 구분선인 경우 그대로 추가
            if para.strip() == "---------------------------------------------------------------------":
                styled_body += para + "\n"
                continue
                
            # <h2> 태그가 포함된 모든 부분 찾기
            matches = h2_pattern.findall(para)
            if matches:
                # 모든 <h2> 태그 내용을 찾아서 처리
                styled_para = para
                for match in matches:
                    # 원래 태그를 포함한 문자열
                    original = f"<h2>{match}</h2>"
                    # 처리 표시로 대체 (나중에 특별 처리할 수 있도록)
                    styled_para = styled_para.replace(original, f"##HEADING_START##{match}##HEADING_END##")
                styled_body += styled_para + "\n"
            else:
                styled_body += para + "\n"

        # 이제 특별 처리 태그를 사용하여 문단 나누기
        paragraphs = []
        for para in styled_body.split('\n'):
            # 구분선인 경우
            if para.strip() == "---------------------------------------------------------------------":
                paragraphs.append((para, "separator"))
                continue
                
            # 특별 태그가 포함된 부분 찾기
            if "##HEADING_START##" in para:
                # 여러 개의 제목이 한 줄에 있을 수 있음
                parts = para.split("##HEADING_START##")
                
                # 첫 부분 처리 (태그 이전 텍스트)
                if parts[0]:
                    paragraphs.append((parts[0], "normal"))
                
                # 나머지 부분 처리
                for i in range(1, len(parts)):
                    if "##HEADING_END##" in parts[i]:
                        heading_parts = parts[i].split("##HEADING_END##")
                        if heading_parts[0]:  # 제목 부분
                            paragraphs.append((heading_parts[0], "heading"))
                        if len(heading_parts) > 1 and heading_parts[1]:  # 제목 이후 부분
                            paragraphs.append((heading_parts[1], "normal"))
            else:
                paragraphs.append((para, "normal"))
        
        # 문단 처리 및 스타일 적용
        for para_text, para_type in paragraphs:
            if para_text.strip() == '':
                # 빈 줄은 간격으로 처리
                story.append(Spacer(1, 0.1 * inch))
            else:
                # 구분선 처리
                if para_type == "separator":
                    story.append(Paragraph(para_text, separator_style))
                    continue
                
                # HTML 예약 문자 처리
                para_text = para_text.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
                
                # 여러 공백을 하나로 처리
                para_text = re.sub(r' +', ' ', para_text)
                
                if para_type == "heading":
                    # 특별 스타일 적용 (큰 폰트 + 볼드 + 밑줄)
                    heading_style = ParagraphStyle(
                        name='Heading2',
                        parent=styles['Normal_Justified'],
                        fontName=font_name,
                        fontSize=14,  # 기본 글씨보다 큰 사이즈
                        fontWeight='bold',  # 볼드처리
                        underline=True,  # 밑줄 추가
                        spaceBefore=16,  # 위 여백 추가
                        spaceAfter=8    # 아래 여백 추가
                    )
                    story.append(Paragraph(para_text, heading_style))
                else:
                    # 일반 문단 추가
                    story.append(Paragraph(para_text, styles['Normal_Justified']))
        
        # PDF 생성
        try:
            doc.build(story)
            print(f"PDF 생성 완료: {output_path}")
        except Exception as e:
            print(f"PDF 생성 실패 ({safe_title}): {e}")
            import traceback
            traceback.print_exc()
            
    except Exception as e:
        print(f"행 처리 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()

print("\n모든 PDF 변환 작업이 완료되었습니다.")

오늘 날짜(20250514) 폴더 생성 완료
시스템 폰트 검색 중...
폰트 발견: /System/Library/Fonts/AppleSDGothicNeo.ttc
폰트 등록 실패(/System/Library/Fonts/AppleSDGothicNeo.ttc): TTC file "/System/Library/Fonts/AppleSDGothicNeo.ttc": postscript outlines are not supported
폰트 발견: /System/Library/Fonts/Supplemental/AppleGothic.ttf
폰트 등록 성공: /System/Library/Fonts/Supplemental/AppleGothic.ttf
엑셀 파일 로드 중: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/post/post_list.xlsx
총 6 개의 행이 로드되었습니다.

처리 중인 행 1/6 - 제목: 5 월 주일 대표 기도문...
PDF 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250514/5 월 주일 대표 기도문_대표 기도문 모음 나눔터.pdf

처리 중인 행 2/6 - 제목: 5월 셋째 주일 대표 기도문...
PDF 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250514/5월 셋째 주일 대표 기도문_대표 기도문 모음 나눔터.pdf

처리 중인 행 3/6 - 제목: 5 월 넷째 주 대표 기도문 모음...
PDF 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250514/5 월 넷째 주 대표 기도문 모음_대표 기도문 모음 나눔터.pdf

처리 중인 행 4/6 - 제목: 6월 첫째 주일 대표 기도문...
PDF 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250514/6월 첫째 주일 대표 기도문_대표

In [5]:
import pandas as pd
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.units import inch
from reportlab.lib.enums import TA_JUSTIFY
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import os
import re
import platform

# 파일 경로 설정
EXCEL_PATH = "/Users/a/Desktop/Work/Blog/2024 Adsense/wp/post/post_list.xlsx"
OUTPUT_DIR = "/Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/"

# 출력 디렉토리가 없으면 생성
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# 시스템에 따른 폰트 경로 설정
print("시스템 폰트 검색 중...")
font_registered = False

# macOS의 경우
if platform.system() == 'Darwin':
    possible_fonts = [
        '/System/Library/Fonts/AppleSDGothicNeo.ttc',
        '/System/Library/Fonts/AppleGothic.ttf',
        '/System/Library/Fonts/Supplemental/AppleGothic.ttf',
        '/Library/Fonts/AppleGothic.ttf',
        '/Library/Fonts/NanumGothic.ttf',
        '/System/Library/Fonts/STHeiti Light.ttc',
        '/System/Library/Fonts/STHeiti Medium.ttc'
    ]
    
    for font_path in possible_fonts:
        if os.path.exists(font_path):
            try:
                # TTF 폰트 등록
                print(f"폰트 발견: {font_path}")
                if font_path.endswith('.ttc'):
                    # TTC(TrueType Collection) 파일 처리
                    pdfmetrics.registerFont(TTFont('KoreanFont', font_path, subfontIndex=0))
                else:
                    pdfmetrics.registerFont(TTFont('KoreanFont', font_path))
                font_registered = True
                print(f"폰트 등록 성공: {font_path}")
                break
            except Exception as e:
                print(f"폰트 등록 실패({font_path}): {e}")
                continue
# Windows의 경우
elif platform.system() == 'Windows':
    possible_fonts = [
        'C:\\Windows\\Fonts\\malgun.ttf',
        'C:\\Windows\\Fonts\\gulim.ttc',
        'C:\\Windows\\Fonts\\batang.ttc',
        'C:\\Windows\\Fonts\\NanumGothic.ttf'
    ]
    
    for font_path in possible_fonts:
        if os.path.exists(font_path):
            try:
                pdfmetrics.registerFont(TTFont('KoreanFont', font_path))
                font_registered = True
                print(f"폰트 등록 성공: {font_path}")
                break
            except Exception as e:
                print(f"폰트 등록 실패({font_path}): {e}")
                continue

# 폰트 등록에 실패한 경우 경고
if not font_registered:
    print("경고: 한글 폰트를 등록할 수 없었습니다. 한글이 제대로 표시되지 않을 수 있습니다.")
    # 기본 폰트 설정
    font_name = 'Helvetica'
else:
    font_name = 'KoreanFont'

# 스타일 설정
styles = getSampleStyleSheet()
styles.add(ParagraphStyle(
    name='Normal_Justified',
    parent=styles['Normal'],
    fontName=font_name,
    alignment=TA_JUSTIFY,
    firstLineIndent=20,
    spaceBefore=12,
    spaceAfter=12,
    leading=14  # 줄 간격
))

# 엑셀 파일 로드
try:
    print(f"엑셀 파일 로드 중: {EXCEL_PATH}")
    df = pd.read_excel(EXCEL_PATH)
    print(f"총 {len(df)} 개의 행이 로드되었습니다.")
except Exception as e:
    print(f"엑셀 파일 로드 실패: {e}")
    import sys
    sys.exit(1)

# 각 행에 대해 PDF 생성
for index, row in df.iterrows():
    try:
        title = str(row['title']).strip()
        body = str(row['body'])
        # HTML 태그 제거 (<h2> 및 </h2>)
        body = body.replace('<h2>', '').replace('</h2>', '').replace('#', '').replace('<h3>', '').replace('</h3>', '')      
        # 디버깅 정보 출력
        print(f"\n처리 중인 행 {index+1}/{len(df)} - 제목: {title[:30]}...")
        
        # 파일명에 사용할 수 없는 문자 제거
        safe_title = re.sub(r'[\\/*?:"<>|]', '', title)
        
        # 출력 파일 경로
        output_path = os.path.join(OUTPUT_DIR, f"{safe_title}.pdf")
        
        # PDF 문서 생성
        doc = SimpleDocTemplate(
            output_path,
            pagesize=A4,
            rightMargin=72,  # 1 inch
            leftMargin=72,
            topMargin=72,
            bottomMargin=72
        )
        
        # 내용을 담을 리스트
        story = []
        
        # 제목 추가 (더 크고 굵게)
        title_style = ParagraphStyle(
            name='Title',
            parent=styles['Title'],
            fontName=font_name,
            fontSize=16,
            spaceAfter=24
        )
        
        # HTML 예약 문자 처리
        title_safe = title.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
        story.append(Paragraph(title_safe, title_style))
        story.append(Spacer(1, 0.2 * inch))
        
        # 본문 내용에서 줄바꿈 처리
        paragraphs = body.split('\n')
        
        for para in paragraphs:
            if para.strip() == '':
                # 빈 줄은 간격으로 처리
                story.append(Spacer(1, 0.1 * inch))
            else:
                # 문단 텍스트에 HTML 태그가 포함된 경우 처리
                para = para.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
                
                # 여러 공백을 하나로 처리 (HTML에서 여러 공백은 하나로 취급됨)
                para = re.sub(r' +', ' ', para)
                
                # 문단 추가 
                story.append(Paragraph(para, styles['Normal_Justified']))
        
        # PDF 생성
        try:
            doc.build(story)
            print(f"PDF 생성 완료: {output_path}")
        except Exception as e:
            print(f"PDF 생성 실패 ({safe_title}): {e}")
            import traceback
            traceback.print_exc()
            
    except Exception as e:
        print(f"행 처리 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()

print("\n모든 PDF 변환 작업이 완료되었습니다.")

시스템 폰트 검색 중...
폰트 발견: /System/Library/Fonts/AppleSDGothicNeo.ttc
폰트 등록 실패(/System/Library/Fonts/AppleSDGothicNeo.ttc): TTC file "/System/Library/Fonts/AppleSDGothicNeo.ttc": postscript outlines are not supported
폰트 발견: /System/Library/Fonts/Supplemental/AppleGothic.ttf
폰트 등록 성공: /System/Library/Fonts/Supplemental/AppleGothic.ttf
엑셀 파일 로드 중: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/post/post_list.xlsx
총 3 개의 행이 로드되었습니다.

처리 중인 행 1/3 - 제목: 창립 주일 대표 기도문...
PDF 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/창립 주일 대표 기도문.pdf

처리 중인 행 2/3 - 제목: 주일학교 헌금 기도문...
PDF 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/주일학교 헌금 기도문.pdf

처리 중인 행 3/3 - 제목: 주일 학교 예배 기도문...
PDF 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/주일 학교 예배 기도문.pdf

모든 PDF 변환 작업이 완료되었습니다.


In [10]:
import pandas as pd
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.units import inch
from reportlab.lib.enums import TA_JUSTIFY
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import os
import re
import platform

# 파일 경로 설정
EXCEL_PATH = "/Users/a/Desktop/Work/Blog/2024 Adsense/wp/post/post_list.xlsx"
OUTPUT_DIR = "/Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/"

# 출력 디렉토리가 없으면 생성
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# 시스템에 따른 폰트 경로 설정
print("시스템 폰트 검색 중...")
font_registered = False

# macOS의 경우
if platform.system() == 'Darwin':
    possible_fonts = [
        '/System/Library/Fonts/AppleSDGothicNeo.ttc',
        '/System/Library/Fonts/AppleGothic.ttf',
        '/System/Library/Fonts/Supplemental/AppleGothic.ttf',
        '/Library/Fonts/AppleGothic.ttf',
        '/Library/Fonts/NanumGothic.ttf',
        '/System/Library/Fonts/STHeiti Light.ttc',
        '/System/Library/Fonts/STHeiti Medium.ttc'
    ]
    
    for font_path in possible_fonts:
        if os.path.exists(font_path):
            try:
                # TTF 폰트 등록
                print(f"폰트 발견: {font_path}")
                if font_path.endswith('.ttc'):
                    # TTC(TrueType Collection) 파일 처리
                    pdfmetrics.registerFont(TTFont('KoreanFont', font_path, subfontIndex=0))
                else:
                    pdfmetrics.registerFont(TTFont('KoreanFont', font_path))
                font_registered = True
                print(f"폰트 등록 성공: {font_path}")
                break
            except Exception as e:
                print(f"폰트 등록 실패({font_path}): {e}")
                continue
# Windows의 경우
elif platform.system() == 'Windows':
    possible_fonts = [
        'C:\\Windows\\Fonts\\malgun.ttf',
        'C:\\Windows\\Fonts\\gulim.ttc',
        'C:\\Windows\\Fonts\\batang.ttc',
        'C:\\Windows\\Fonts\\NanumGothic.ttf'
    ]
    
    for font_path in possible_fonts:
        if os.path.exists(font_path):
            try:
                pdfmetrics.registerFont(TTFont('KoreanFont', font_path))
                font_registered = True
                print(f"폰트 등록 성공: {font_path}")
                break
            except Exception as e:
                print(f"폰트 등록 실패({font_path}): {e}")
                continue

# 폰트 등록에 실패한 경우 경고
if not font_registered:
    print("경고: 한글 폰트를 등록할 수 없었습니다. 한글이 제대로 표시되지 않을 수 있습니다.")
    # 기본 폰트 설정
    font_name = 'Helvetica'
else:
    font_name = 'KoreanFont'

# 스타일 설정
styles = getSampleStyleSheet()
styles.add(ParagraphStyle(
    name='Normal_Justified',
    parent=styles['Normal'],
    fontName=font_name,
    alignment=TA_JUSTIFY,
    firstLineIndent=20,
    spaceBefore=12,
    spaceAfter=12,
    leading=14  # 줄 간격
))

# 엑셀 파일 로드
try:
    print(f"엑셀 파일 로드 중: {EXCEL_PATH}")
    df = pd.read_excel(EXCEL_PATH)
    print(f"총 {len(df)} 개의 행이 로드되었습니다.")
except Exception as e:
    print(f"엑셀 파일 로드 실패: {e}")
    import sys
    sys.exit(1)

# 각 행에 대해 PDF 생성
for index, row in df.iterrows():
    try:
        title = str(row['title']).strip()
        body = str(row['body'])
        # HTML 태그 제거 (<h2> 및 </h2>)
        body = body.replace('<h3>', '').replace('</h3>', '').replace('#', '')       
        # 디버깅 정보 출력
        print(f"\n처리 중인 행 {index+1}/{len(df)} - 제목: {title[:30]}...")
        
        # 파일명에 사용할 수 없는 문자 제거
        safe_title = re.sub(r'[\\/*?:"<>|]', '', title)
        
        # 출력 파일 경로
        output_path = os.path.join(OUTPUT_DIR, f"{safe_title}.pdf")
        
        # PDF 문서 생성
        doc = SimpleDocTemplate(
            output_path,
            pagesize=A4,
            rightMargin=72,  # 1 inch
            leftMargin=72,
            topMargin=72,
            bottomMargin=72
        )
        
        # 내용을 담을 리스트
        story = []
        
        # 제목 추가 (더 크고 굵게)
        title_style = ParagraphStyle(
            name='Title',
            parent=styles['Title'],
            fontName=font_name,
            fontSize=16,
            spaceAfter=24
        )
        
        # HTML 예약 문자 처리
        title_safe = title.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
        story.append(Paragraph(title_safe, title_style))
        story.append(Spacer(1, 0.2 * inch))
        
        # <h2> 태그를 찾는 정규 표현식
        h2_pattern = re.compile(r'<h2>(.*?)</h2>')

        # 본문에서 <h2> 태그를 찾아 스타일 적용
        styled_body = ""
        for para in body.split('\n'):
            # <h2> 태그가 포함된 모든 부분 찾기
            matches = h2_pattern.findall(para)
            if matches:
                # 모든 <h2> 태그 내용을 찾아서 처리
                styled_para = para
                for match in matches:
                    # 원래 태그를 포함한 문자열
                    original = f"<h2>{match}</h2>"
                    # 처리 표시로 대체 (나중에 특별 처리할 수 있도록)
                    styled_para = styled_para.replace(original, f"##HEADING_START##{match}##HEADING_END##")
                styled_body += styled_para + "\n"
            else:
                styled_body += para + "\n"

        # 이제 특별 처리 태그를 사용하여 문단 나누기
        paragraphs = []
        for para in styled_body.split('\n'):
            # 특별 태그가 포함된 부분 찾기
            if "##HEADING_START##" in para:
                # 여러 개의 제목이 한 줄에 있을 수 있음
                parts = para.split("##HEADING_START##")
                
                # 첫 부분 처리 (태그 이전 텍스트)
                if parts[0]:
                    paragraphs.append((parts[0], False))
                
                # 나머지 부분 처리
                for i in range(1, len(parts)):
                    if "##HEADING_END##" in parts[i]:
                        heading_parts = parts[i].split("##HEADING_END##")
                        if heading_parts[0]:  # 제목 부분
                            paragraphs.append((heading_parts[0], True))
                        if len(heading_parts) > 1 and heading_parts[1]:  # 제목 이후 부분
                            paragraphs.append((heading_parts[1], False))
            else:
                paragraphs.append((para, False))
        
        # 문단 처리 및 스타일 적용
        for para_text, is_heading in paragraphs:
            if para_text.strip() == '':
                # 빈 줄은 간격으로 처리
                story.append(Spacer(1, 0.1 * inch))
            else:
                # HTML 예약 문자 처리
                para_text = para_text.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
                
                # 여러 공백을 하나로 처리
                para_text = re.sub(r' +', ' ', para_text)
                
                if is_heading:
                    # 특별 스타일 적용 (큰 폰트 + 볼드 + 밑줄)
                    heading_style = ParagraphStyle(
                        name='Heading2',
                        parent=styles['Normal_Justified'],
                        fontName=font_name,
                        fontSize=14,  # 기본 글씨보다 큰 사이즈
                        fontWeight='bold',  # 볼드처리
                        underline=True,  # 밑줄 추가
                        spaceBefore=16,  # 위 여백 추가
                        spaceAfter=8    # 아래 여백 추가
                    )
                    story.append(Paragraph(para_text, heading_style))
                else:
                    # 일반 문단 추가
                    story.append(Paragraph(para_text, styles['Normal_Justified']))
        
        # PDF 생성
        try:
            doc.build(story)
            print(f"PDF 생성 완료: {output_path}")
        except Exception as e:
            print(f"PDF 생성 실패 ({safe_title}): {e}")
            import traceback
            traceback.print_exc()
            
    except Exception as e:
        print(f"행 처리 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()

print("\n모든 PDF 변환 작업이 완료되었습니다.")

시스템 폰트 검색 중...
폰트 발견: /System/Library/Fonts/AppleSDGothicNeo.ttc
폰트 등록 실패(/System/Library/Fonts/AppleSDGothicNeo.ttc): TTC file "/System/Library/Fonts/AppleSDGothicNeo.ttc": postscript outlines are not supported
폰트 발견: /System/Library/Fonts/Supplemental/AppleGothic.ttf
폰트 등록 성공: /System/Library/Fonts/Supplemental/AppleGothic.ttf
엑셀 파일 로드 중: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/post/post_list.xlsx
총 3 개의 행이 로드되었습니다.

처리 중인 행 1/3 - 제목: 창립 주일 대표 기도문...
PDF 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/창립 주일 대표 기도문.pdf

처리 중인 행 2/3 - 제목: 주일학교 헌금 기도문...
PDF 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/주일학교 헌금 기도문.pdf

처리 중인 행 3/3 - 제목: 주일 학교 예배 기도문...
PDF 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/주일 학교 예배 기도문.pdf

모든 PDF 변환 작업이 완료되었습니다.
